In [2]:
import h2o
import pandas
from h2o.automl import H2OAutoML

h2o.init()

breast_tissue = h2o.import_file("BreastTissue.csv")

train, valid = breast_tissue.split_frame(ratios=[.8], seed=1)

x = train.columns
y = "Class"
x.remove(y)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,11 secs
H2O_cluster_timezone:,Europe/Belgrade
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.5
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_lechw_hgtkey
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,238.4 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train, validation_frame=valid)

AutoML progress: |██
15:48:18.62: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
15:48:18.77: AutoML: XGBoost is not available; skipping it.

████████
15:48:26.173: Skipping training of model GBM_5_AutoML_20200629_154818 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200629_154818.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 91.0.


██████████████████████████████████████████████| 100%


In [4]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
GBM_grid__1_AutoML_20200629_154818_model_4,0.0461538,0.223264,0.234313,0.0549027
GBM_3_AutoML_20200629_154818,0.0554131,0.22278,0.233845,0.0546835
GBM_4_AutoML_20200629_154818,0.0600427,0.236244,0.235211,0.0553244
GBM_2_AutoML_20200629_154818,0.069302,0.215687,0.231168,0.0534384
GBM_1_AutoML_20200629_154818,0.069302,0.22053,0.236156,0.0557696
GBM_grid__1_AutoML_20200629_154818_model_6,0.0878205,0.243575,0.240695,0.0579342
DeepLearning_grid__3_AutoML_20200629_154818_model_1,0.0915242,0.980789,0.2955,0.08732
XRT_1_AutoML_20200629_154818,0.0932336,0.399387,0.341233,0.11644
GBM_grid__1_AutoML_20200629_154818_model_1,0.0960114,0.271713,0.265733,0.0706143
GBM_grid__1_AutoML_20200629_154818_model_2,0.101567,0.349751,0.314978,0.0992112


In [29]:
best_model = aml.leader

In [51]:
best_model.save_mojo()

'C:\\Users\\lechw\\Projects\\Asseco\\GBM_grid__1_AutoML_20200629_154818_model_4.zip'

In [49]:
data_frame = best_model.confusion_matrix(breast_tissue).as_data_frame().to_csv("confusion_matrix.csv")

In [105]:
best_model.model_performance(breast_tissue)


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.002096964589768975
RMSE: 0.04579262593222816
LogLoss: 0.022219636391365476
Mean Per-Class Error: 0.0

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,adi,car,con,fad,gla,mas,Error,Rate
0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 22
1,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0 / 21
2,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0 / 14
3,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0 / 15
4,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0 / 16
5,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0 / 18
6,22.0,21.0,14.0,15.0,16.0,18.0,0.0,0 / 106



Top-6 Hit Ratios:


,k,hit_ratio
0,1,1.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,1.0
5,6,1.0
